In [5]:
import pandas as pd
import datetime
import numpy as np
import yfinance as yf
import pytrade
import matplotlib.pyplot as plt

pd.options.display.float_format = '{:,.3f}'.format?

In [6]:
%%time
raw = pytrade.get_data(reload = False)
ratios = pytrade.calculate_ratio(raw)
output = pytrade.analysis_1.run(ratios)
output.sort_values(by="short_term_metric",ascending=False,inplace=True)
my_shortterm = output.loc[["DG","BLK","DXCM","QRVO","AMD","LULU","MSFT"]]
my_longterm = output.loc[["AMGN","AAPL","WMT","OKE","INTC","KLAC","REGN"]]
output

Wall time: 58 ms


,latest,three_wk_high,three_wk_low,three_m_high,three_m_low,one_yr_high,one_yr_low,three_wk_range,three_m_range,one_yr_range,short_term_stochastic,long_term_stochastic,st_potential,lt_potential,short_term_metric,long_term_metric
MELI,972.630,"1,039.889",972.630,"1,039.889",631.424,"1,039.889",501.902,0.000,0.835,0.875,0.875,0.040,0.069,0.069,1.000,0.161
PYPL,172.300,185.873,172.300,185.873,123.342,185.873,102.848,0.000,0.783,0.837,0.837,0.054,0.079,0.079,0.979,0.192
ADSK,234.600,251.780,234.600,251.780,193.487,251.780,149.492,0.000,0.705,0.832,0.832,0.127,0.073,0.073,0.964,0.256
ADBE,433.780,468.027,433.780,468.027,372.360,468.027,278.106,0.000,0.642,0.820,0.820,0.178,0.079,0.079,0.962,0.317
AVGO,315.080,326.664,315.080,326.664,286.415,326.664,217.845,0.000,0.712,0.894,0.894,0.181,0.037,0.037,0.960,0.246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BRK-A,"284,780.000","284,780.000","272,657.292","325,971.585","272,657.292","359,131.903","272,657.292",1.000,0.227,0.140,-0.860,-0.087,-0.043,-0.043,-0.937,-0.162
AEE,76.570,76.573,71.348,87.677,71.348,100.884,71.348,0.999,0.320,0.177,-0.823,-0.143,-0.068,-0.068,-0.946,-0.263
WRK,29.150,29.150,26.598,36.841,26.598,42.001,26.598,1.000,0.249,0.166,-0.834,-0.083,-0.088,-0.088,-0.993,-0.238
L,35.520,35.520,33.293,41.958,32.544,58.200,32.544,1.000,0.316,0.116,-0.884,-0.200,-0.061,-0.084,-0.994,-0.348


In [ ]:
output.sort_values(by="long_term_metric",ascending=False,inplace=True)
output

In [ ]:
my_longterm

In [ ]:
my_shortterm

In [ ]:
plt.close('all')
ratios["DXCM"].tail(250).plot()

In [ ]:
shortterm_portfolio = pytrade.Portfolio.load("st_portfolio", raw)

In [ ]:
longterm_portfolio = pytrade.Portfolio.load("lt_portfolio", raw)

In [ ]:
shortterm_portfolio.save()

In [ ]:
shortterm_portfolio.report()

In [ ]:
shortterm_portfolio.buy_power

In [ ]:
shortterm_portfolio.current_value()

In [7]:
stock = pytrade.Stock("AAPL",2,400)

In [8]:
st = pytrade.StockTransaction(datetime.date.today(), pytrade.TransactionType.BUY, stock)

In [9]:
print(st)

AAPL: 2 @ 200.0


In [10]:
dt = pytrade.DepositTransaction(datetime.date.today(), pytrade.TransactionType.DEPOSIT, 1000)

In [11]:
print(dt)

1000


In [12]:
t = pytrade.Transaction(datetime.date.today(), pytrade.TransactionType.DEPOSIT)

In [17]:
t.value

AttributeError: 'Transaction' object has no attribute 'value'